In [11]:
import keras

/Users/zukako/.pyenv/versions/anaconda3-4.0.0/envs/keras/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [12]:
from keras.layers import Input, Embedding, LSTM, Dense
from keras.models import Model

dataset作成

In [21]:
import random
import numpy as np
headline_data=[]
for j in range(20):
    random_list = []
    for i in range(100):
        random_data = random.randrange(10000)
        random_list.append(random_data)
    headline_data.append(random_list)
headline_data=np.array(headline_data)

In [30]:
additional_data=[]
for j in range(20):
    random_list = []
    for i in range(5):
        random_data = random.randrange(10000)
        random_list.append(random_data)
    additional_data.append(random_list)
additional_data=np.array(additional_data)

In [22]:
labels=[]
for i in range(20):
    labels.append(random.randrange(1000))
labels=np.array(labels)

In [23]:
main_input = Input(shape=(100,), dtype='int32', name='main_input')
x = Embedding(output_dim=512, input_dim=10000, input_length=100)(main_input)
lstm_out = LSTM(32)(x)

lstm_outを監視することでLSTMが訓練しやすくなる。

In [24]:
auxiliary_output = Dense(1, activation='sigmoid', name='aux_output')(lstm_out)

In [25]:
auxiliary_input = Input(shape=(5,), name='aux_input')
# lstm_outとaux_inputを結合したものを次の層への入力とする。
x = keras.layers.concatenate([lstm_out, auxiliary_input])

# We stack a deep densely-connected network on top
x = Dense(64, activation='relu')(x)
x = Dense(64, activation='relu')(x)
x = Dense(64, activation='relu')(x)

# And finally we add the main logistic regression layer
main_output = Dense(1, activation='sigmoid', name='main_output')(x)

モデルの定義。二つの入力と二つの出力を持っている。

In [26]:
model = Model(inputs=[main_input, auxiliary_input], outputs=[main_output, auxiliary_output])

modelをコンパイルする。誤差関数などをリストで渡すとそれぞれそれに応じて評価される。<br>
optimizerやらlossを一つにした場合は共通のものになる。<br>
loss_weights罰則を定義する。

In [27]:
model.compile(optimizer='rmsprop', loss='binary_crossentropy',
              loss_weights=[1., 0.2])

In [31]:
model.fit([headline_data, additional_data], [labels, labels],
          epochs=50, batch_size=12)

Epoch 1/50
20/20 [==============================] - 2s 79ms/step - loss: 5493.1309 - main_output_loss: 5492.9466 - aux_output_loss: 0.9199
Epoch 2/50
20/20 [==============================] - 0s 17ms/step - loss: 5479.1462 - main_output_loss: 5492.9468 - aux_output_loss: -69.0031
Epoch 3/50
20/20 [==============================] - 0s 19ms/step - loss: 5461.7197 - main_output_loss: 5492.9468 - aux_output_loss: -156.1354
Epoch 4/50
20/20 [==============================] - 0s 20ms/step - loss: 5427.7913 - main_output_loss: 5492.9472 - aux_output_loss: -325.7785
Epoch 5/50
20/20 [==============================] - 0s 18ms/step - loss: 5360.9702 - main_output_loss: 5492.9470 - aux_output_loss: -659.8841
Epoch 6/50
20/20 [==============================] - 0s 19ms/step - loss: 5282.9805 - main_output_loss: 5492.9468 - aux_output_loss: -1049.8319
Epoch 7/50
20/20 [==============================] - 0s 18ms/step - loss: 5228.5674 - main_output_loss: 5492.9473 - aux_output_loss: -1321.8992
Epoch 8/

入力と出力に重み付けを行なっていれば以下のように名前で定義できる・

In [ ]:
model.compile(optimizer='rmsprop',
              loss={'main_output': 'binary_crossentropy', 'aux_output': 'binary_crossentropy'},
              loss_weights={'main_output': 1., 'aux_output': 0.2})

# And trained it via:
model.fit({'main_input': headline_data, 'aux_input': additional_data},
          {'main_output': labels, 'aux_output': labels},
          epochs=50, batch_size=32)